/root


In [14]:
import sys
import os
import json
import time
import tensorflow as tf
import numpy as np
import glob
from PIL import Image

print(sys.path)
from ..lib.utility import load_config
from object_detection.utils import label_map_util


PATH_OUTPUT = '../output_b4_with_momentum/bdd100k_test.json'
PATH_TO_CKPT = '../output_b4_with_momentum/frozen_inference_graph.pb'
PATH_TEST_IDS = '/workspace/datasets/BDD100k/VOC_test/ImageSets/Main/test.txt'
DIR_IMAGE = '/workspace/datasets/BDD100k/bdd100k/images/100k/test/'
PATH_TO_LABELS = '../label/label_map.pbtxtx'
NUM_CLASSES = 7

def readCfg(path="../cfg/model_inference.json"):
    try:
        with open(path, 'r') as cfg_fd:
            json_fd = json.load(cfg_fd)
            pathDelSlash(json_fd)
            print(json_fd)
            return json_fd
    except:
        raise("Config file open failed")

def pathDelSlash(json_file):
    for i in json_file:
        if isinstance(json_file[i], str):
            if '/' in json_file[i]:
                if json_file[i][-1] == '/':
                    json_file[i] = json_file[i][:-1]
        else:
            pass
        
readCfg()
asdsd

def get_results(boxes, classes, scores, category_index, im_width, im_height,
    min_score_thresh=.2):
    bboxes = list()
    for i, box in enumerate(boxes):
        if scores[i] > min_score_thresh:
            ymin, xmin, ymax, xmax = box
            bbox = {
                'bbox': {
                    'xmax': xmax * im_width,
                    'xmin': xmin * im_width,
                    'ymax': ymax * im_height,
                    'ymin': ymin * im_height
                },
                'category': category_index[classes[i]]['name'],
                'score': float(scores[i])
            }
            bboxes.append(bbox)
    return bboxes
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
total_time = 0
test_annos = dict()
flag = False
with detection_graph.as_default():
    with tf.Session(graph=detection_graph, config=config) as sess:
        for index, image_path in enumerate(glob.glob(DIR_IMAGE+'*.jpg')):
            image_id = image_path.rstrip().split('/')[-1]
            image = Image.open(image_path)
            image_np = np.array(image).astype(np.uint8)
            im_height, im_width, _ = image_np.shape
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            start_time = time.time()
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            end_time = time.time()
            print('{} {} {:.3f}s'.format(time.ctime(), image_id, end_time - start_time))
            if flag:
                total_time += end_time - start_time
            else:
                flag = True
            test_annos[image_id] = {'objects': get_results(
                np.squeeze(boxes), np.squeeze(classes).astype(np.int32), np.squeeze(scores), category_index,
                im_width, im_height)}
            if index > 10:
                break
                
test_annos = {'imgs': test_annos}
fd = open(PATH_OUTPUT, 'w')
json.dump(test_annos, fd)
fd.close()
print("success")

['/opt/intel/intelpython3/lib/python36.zip', '/opt/intel/intelpython3/lib/python3.6', '/opt/intel/intelpython3/lib/python3.6/lib-dynload', '', '/root/.local/lib/python3.6/site-packages', '/opt/intel/intelpython3/lib/python3.6/site-packages', '/opt/intel/intelpython3/lib/python3.6/site-packages/pycocotools-2.0-py3.6-linux-x86_64.egg', '/opt/intel/intelpython3/lib/python3.6/site-packages/html5lib-1.0.1-py3.6.egg', '/opt/intel/intelpython3/lib/python3.6/site-packages/pip-9.0.3-py3.6.egg', '/opt/intel/intelpython3/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/workspace/yo/tensorflow_obj_detect_api/google_obj_detection/research', '/workspace/yo/tensorflow_obj_detect_api/google_obj_detection/research/slim', '/workspace/yo/tensorflow_obj_detect_api/google_obj_detection/research', '/workspace/yo/tensorflow_obj_detect_api/google_obj_detection/research/slim', '/workspace/yo/tensorflow_obj_detect_api/google_obj_detection/research', '/workspace/yo/tensorflow_obj_detect_api/g

ValueError: attempted relative import beyond top-level package